In [1]:
#import

import re
import json
import glob
import os

<h2>AFD e Lexer</h2>
<h4>Nessa parte, definimos a classe de tokens, a classe de AFDs, criamos o afd e por fim definimos a classe lexer</h4>

In [2]:
#class token

class Token:
    def __init__(self, value, type, line):
        self.value = value
        self.type = type
        self.line = line
    
    def __str__(self):
        return f"Token({self.type}, '{self.value}', line {self.line})"
    
    def to_dict(self):
        return {
            "token": self.type,
            "linha": self.line,
            "lexema": self.value
        }

In [3]:
#class afd
class AFD:
    def __init__(self, name, transitions, start, accepts):
        self.name = name
        self.transitions = transitions
        self.start = start
        self.accepts = accepts
        self.reset()

    def reset(self):
        self.state = self.start
        self.lexeme = ""

    def step(self, c):
        trans = self.transitions.get(self.state, {})
        if c in trans:
            self.state = trans[c]
        elif 'LETTER' in trans and c.isalpha():
            self.state = trans['LETTER']
        elif 'DIGIT' in trans and c.isdigit():
            self.state = trans['DIGIT']
        elif 'ANY' in trans and c != '"' and c != "'" and c != '\n':
            self.state = trans['ANY']
        else:
            return False
        self.lexeme += c
        return True

    def is_accepting(self):
        return self.state in self.accepts

    def token_type(self):
        return self.accepts.get(self.state)

In [4]:
#make afd
def make_afd():
    transitions = {
        # Estado inicial (0)
        0: {
            'LETTER': 1, 'DIGIT': 2, '_': 1,
            '=': 3, '!': 4, '>': 5, '<': 6, '-': 7,
            '+': 8, '*': 9, '/': 10,
            '(': 11, ')': 12, '{': 13, '}': 14,
            ';': 15, ',': 16, ':': 17,
            '"': 18, "'": 19
        },

        # Identificadores (1)
        1: {'LETTER': 1, 'DIGIT': 1, '_': 1},

        # Números inteiros (2)
        2: {'DIGIT': 2, '.': 20},
        
        # Números float (20 -> 21)
        20: {'DIGIT': 21},
        21: {'DIGIT': 21},

        # Operadores relacionais
        3: {'=': 22},   # = -> ==
        4: {'=': 23},   # ! -> !=
        5: {'=': 24},   # > -> >=
        6: {'=': 25},   # < -> <=
        7: {'>': 26},   # - -> ->

        # Strings (18)
        18: {'ANY': 18, '"': 27},

        # Char literals (19 -> 28 -> 29)
        19: {'ANY': 28},
        28: {"'": 29},
    }

    accepts = {
        1: "ID",
        2: "INT_CONST",
        21: "FLOAT_CONST",
        3: "ASSIGN",
        7: "MINUS",
        8: "PLUS",
        9: "MULT",
        10: "DIV",  
        22: "EQ",
        23: "NE", 
        24: "GE",
        25: "LE",
        26: "ARROW",
        5: "GT",
        6: "LT",
        11: "LBRACKET",
        12: "RBRACKET",
        13: "LBRACE",
        14: "RBRACE",
        15: "SEMICOLON",
        16: "COMMA",
        17: "COLON",
        27: "FMT_STRING",
        29: "CHAR_LITERAL",
    }

    return AFD("global", transitions, 0, accepts)

In [5]:
#class lexer
class Lexer:
    def __init__(self, source):
        self.source = source
        self.tokens = []
        self.afd = make_afd()

    def tokenize(self):
        i = 0
        line = 1
        
        while i < len(self.source):
            c = self.source[i]

            # Controle de linhas
            if c == '\n':
                line += 1
                i += 1
                continue

            # Ignorar espaços em branco
            if c.isspace():
                i += 1
                continue

            afd = self.afd
            afd.reset()
            j = i
            
            # Processar lexema com AFD
            while j < len(self.source) and afd.step(self.source[j]):
                j += 1

            # Verificar se reconheceu um token
            if afd.is_accepting():
                lex = afd.lexeme
                tipo = afd.token_type()

                # Palavras reservadas
                reserved = {
                    "fn": "FUNCTION", "main": "MAIN", "let": "LET", 
                    "int": "INT", "float": "FLOAT", "char": "CHAR", 
                    "if": "IF", "else": "ELSE", "while": "WHILE", 
                    "println": "PRINTLN", "return": "RETURN"
                }
                
                if lex in reserved:
                    tipo = reserved[lex]

                self.tokens.append(Token(lex, tipo, line))
                i += len(lex)
            else:
                print(f"Erro léxico: caractere inesperado '{self.source[i]}' na linha {line}")
                i += 1
        self.tokens.append(Token("", "EOF", line))
        return self.tokens

<h2>Testes do Lexer</h2>
<h4>Aqui executamos os testes referentes a primeira entrega, onde são usados arquivos de teste da pasta "entradas de tokens reconhecidos" e é gerado um json para cada teste que é salvo na pasta "saidas de tokens reconhecidos"</h4>

In [6]:
#teste calculadora.p
entrada = "codigos base da linguagem p/calculadora.p"
saida = "saidas de tokens reconhecidos/calculadora_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

In [7]:
#teste lexical_error.p
entrada = "codigos base da linguagem p/lexical_error.p"
saida = "saidas de tokens reconhecidos/lexical_error_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

Erro léxico: caractere inesperado '[' na linha 3
Erro léxico: caractere inesperado ']' na linha 3
Erro léxico: caractere inesperado '$' na linha 5
Erro léxico: caractere inesperado '&' na linha 6


In [8]:
#teste loop_simples.p
entrada = "codigos base da linguagem p/loop_simples.p"
saida = "saidas de tokens reconhecidos/loop_simples_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

In [9]:
#teste media.p
entrada = "codigos base da linguagem p/media.p"
saida = "saidas de tokens reconhecidos/media_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

In [10]:
#teste soma.p
entrada = "codigos base da linguagem p/soma.p"
saida = "saidas de tokens reconhecidos/soma_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

In [11]:
#teste tokens.p
entrada = "codigos base da linguagem p/tokens.p"
saida = "saidas de tokens reconhecidos/tokens_tokens.json"

lexer = Lexer(open(entrada, "r", encoding="utf-8").read())
tokens = lexer.tokenize()

tokens_json = [token.to_dict() for token in tokens]

with open(saida, "w", encoding="utf-8") as f:
    json.dump(tokens_json, f, indent=2, ensure_ascii=False)

<h2>Parser e Tabela de Simbolos</h2>

In [12]:
#class parser
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0
        self.current_token = tokens[0] if tokens else None
        self.errors = []
        self.symbol_table_manager = SymbolTableManager()
        self.current_function = None
        self.param_position = 0

    def parse(self, nome_arquivo=None):
        try:
            self.parse_programa()
            if self.current_token and self.current_token.type != "EOF":
                self.error("Esperado fim de arquivo")
        except Exception as e:
            self.error(f"Erro de sintaxe: {str(e)}")
        
        self.generate_output_files(nome_arquivo)

    def advance(self):
        self.pos += 1
        if self.pos < len(self.tokens):
            self.current_token = self.tokens[self.pos]
        else:
            self.current_token = None
    
    def match(self, expected_type):
        if self.current_token is None:
            self.error(f"Esperado '{expected_type}', encontrado 'EOF'")
            return None
        if self.current_token.type == expected_type:
            token = self.current_token
            self.advance()
            return token
        else:
            found = self.current_token.type 
            self.error(f"Esperado '{expected_type}', encontrado '{found}'")
            return None
    
    def error(self, message):
        if self.current_token is not None:
            line = self.current_token.line
        elif self.tokens:
            line = self.tokens[-1].line
        else:
            line = 1
        
        error_message = f"Linha {line}: {message}"
        self.errors.append(error_message)
        print(f"Erro de sintaxe: {error_message}")
    
    # ========== MÉTODOS DE RECUPERAÇÃO DE ERROS ==========
    def sync_to(self, sync_tokens):
        """Recuperação de erro - pula tokens até encontrar um dos sync_tokens"""
        while (self.current_token and 
               self.current_token.type not in sync_tokens and
               self.current_token.type != "EOF"):
            print(f"  [RECUPERAÇÃO] Pulando token: {self.current_token}")
            self.advance()
        
        if self.current_token and self.current_token.type in sync_tokens:
            print(f"  [RECUPERAÇÃO] Sincronizado em: {self.current_token}")
            return True
        return False

    def match_with_recovery(self, expected_type, sync_tokens=None):
        """Match com recuperação de erro"""
        if sync_tokens is None:
            sync_tokens = ["SEMICOLON", "RBRACE", "FUNCTION", "EOF"]
            
        if self.current_token is None:
            self.error(f"Esperado '{expected_type}', encontrado 'EOF'")
            return None
            
        if self.current_token.type == expected_type:
            token = self.current_token
            self.advance()
            return token
        else:
            found = self.current_token.type 
            self.error(f"Esperado '{expected_type}', encontrado '{found}'")
            # Recuperação
            self.sync_to(sync_tokens)
            return None
    
    # ========== PRODUÇÕES DA GRAMÁTICA ==========
    
    def parse_programa(self):
        self.parse_funcao()
        self.parse_funcao_seq()
    
    def parse_funcao_seq(self):
        while self.current_token and self.current_token.type == "FUNCTION":
            self.parse_funcao()

    def parse_funcao(self):
        self.match_with_recovery("FUNCTION")
        
        nome_funcao_token = self.parse_nome_funcao()
        if nome_funcao_token:
            nome_funcao = nome_funcao_token.value
            self.current_function = nome_funcao
            self.symbol_table_manager.enter_scope(nome_funcao)
            
            # Adicionar função como símbolo
            func_symbol = Symbol(nome_funcao, "function", False, nome_funcao_token.line, nome_funcao)
            self.symbol_table_manager.current_table.add_symbol(func_symbol)
        
        self.match_with_recovery("LBRACKET")
        self.param_position = 0  # Reset contador de parâmetros
        self.parse_lista_params()
        self.match_with_recovery("RBRACKET")
        
        # Processar tipo de retorno
        return_type = self.parse_tipo_retorno_funcao()
        if return_type:
            self.symbol_table_manager.current_table.set_return_type(return_type)
        
        self.parse_bloco()
        
        self.symbol_table_manager.exit_scope()
        self.current_function = None
    
    def parse_nome_funcao(self):
        if self.current_token and self.current_token.type in ["MAIN", "ID"]:
            return self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado nome de função (ID ou 'main')")
            return None
    
    def parse_lista_params(self):
        if self.current_token and self.current_token.type == "ID":
            param_name_token = self.match_with_recovery("ID")
            self.match_with_recovery("COLON")
            type_token = self.parse_type()

            if param_name_token and type_token and self.symbol_table_manager.current_table:
                # ADICIONAR parâmetro com posição correta
                param_symbol = Symbol(
                    param_name_token.value, 
                    type_token.value.lower(), 
                    True,  # is_param = True
                    param_name_token.line,
                    self.current_function,
                    self.param_position  # posição atual
                )
                if not self.symbol_table_manager.current_table.add_symbol(param_symbol):
                    self.error(f"Parâmetro '{param_name_token.value}' já declarado")
                
                self.param_position += 1  # Incrementar posição
            
            self.parse_lista_params2()
    
    def parse_lista_params2(self):
        while self.current_token and self.current_token.type == "COMMA":
            self.match_with_recovery("COMMA")
            param_name_token = self.match_with_recovery("ID")
            self.match_with_recovery("COLON")
            type_token = self.parse_type()
            
            if param_name_token and type_token and self.symbol_table_manager.current_table:
                param_symbol = Symbol(
                    param_name_token.value, 
                    type_token.value.lower(), 
                    True,  # is_param = True
                    param_name_token.line,
                    self.current_function,
                    self.param_position  # posição atual
                )
                if not self.symbol_table_manager.current_table.add_symbol(param_symbol):
                    self.error(f"Parâmetro '{param_name_token.value}' já declarado")
                
                self.param_position += 1  # Incrementar posição
    
    def parse_tipo_retorno_funcao(self):
        if self.current_token and self.current_token.type == "ARROW":
            self.match_with_recovery("ARROW")
            type_token = self.parse_type()
            if type_token:
                return type_token.value.lower()
        return None

    def parse_type(self):
        if self.current_token and self.current_token.type in ["INT", "FLOAT", "CHAR"]:
            return self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado tipo de dado (int, float ou char)")
            return None
    
    def parse_bloco(self):
        self.match_with_recovery("LBRACE")
        self.parse_sequencia()
        self.match_with_recovery("RBRACE")
    
    def parse_sequencia(self):
        while self.current_token and self.current_token.type in self._first_declaracao_comando():
            if self._is_declaracao():
                self.parse_declaracao()
            else:
                self.parse_comando()
    
    def _first_declaracao_comando(self):
        return ["LET", "ID", "IF", "WHILE", "PRINTLN", "RETURN"]
    
    def _is_declaracao(self):
        return self.current_token and self.current_token.type == "LET"
    
    # ========== SISTEMA DE SAÍDA ==========
    def generate_output_files(self, nome_arquivo=None):
        """Gera arquivos de saída unificados"""
        self._generate_syntax_errors(nome_arquivo)
        self._generate_symbol_tables_txt(nome_arquivo)

    def _generate_syntax_errors(self, nome_arquivo=None): 
        try:
            os.makedirs("saidas do analisador sintatico", exist_ok=True)
            
            if nome_arquivo is None:
                nome_base = "syntax_errors"
            else:
                nome_base = os.path.splitext(nome_arquivo)[0]
            
            # Arquivo TXT para erros sintáticos
            txt_path = f"saidas do analisador sintatico/{nome_base}_syntax.txt"
            
            with open(txt_path, "w", encoding="utf-8") as f:
                if len(self.errors) == 0:
                    f.write(f"Arquivo: {nome_arquivo or 'desconhecido'}\n")
                    f.write("Status: VÁLIDO\n")
                    f.write("Não foram encontrados erros sintáticos.\n")
                else:
                    f.write(f"Arquivo: {nome_arquivo or 'desconhecido'}\n")
                    f.write(f"Status: INVÁLIDO - {len(self.errors)} erro(s) encontrado(s)\n\n")
                    f.write("ERROS SINTÁTICOS:\n")
                    f.write("=" * 50 + "\n")
                    for erro in self.errors:
                        f.write(f"{erro}\n")
                        
        except Exception as e:
            print(f"Erro ao gerar arquivo de erros de sintaxe: {str(e)}")

    def _generate_symbol_tables_txt(self, nome_arquivo=None):
        """Gera arquivo TXT com tabelas de símbolos formatadas"""
        try:
            os.makedirs("tabelas de simbolos", exist_ok=True)
            
            if nome_arquivo is None:
                nome_base = "symbol_tables"
            else:
                nome_base = os.path.splitext(nome_arquivo)[0]
            
            txt_path = f"tabelas de simbolos/{nome_base}_tabelas.txt"
            
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(f"ARQUIVO: {nome_arquivo or 'desconhecido'}\n")
                f.write("=" * 80 + "\n\n")
                
                for table in self.symbol_table_manager.tables:
                    # Cabeçalho da tabela
                    f.write(f"TABELA DE SÍMBOLOS: {table.scope_name}\n")
                    f.write("-" * 80 + "\n")
                    f.write(f"{'Chave':<15} {'Nome':<15} {'Tipo':<12} {'Parâmetro':<10} {'Pos_Param':<10} {'Chamadas'}\n")
                    f.write("-" * 80 + "\n")
                    
                    # Linhas da tabela
                    for symbol_name, symbol in table.symbols.items():
                        is_param_str = "Sim" if symbol.is_param else "Não"
                        pos_param_str = str(symbol.pos_param) if symbol.is_param else "-1"
                        
                        # Formatar chamadas de função
                        chamadas_str = "NULL"
                        if symbol.call_refs:
                            chamadas_list = []
                            for call in symbol.call_refs:
                                chamadas_list.append(f"{call.name}({', '.join(call.args)})")
                            chamadas_str = ", ".join(chamadas_list)
                        
                        f.write(f"{symbol_name:<15} {symbol.name:<15} {symbol.datatype:<12} {is_param_str:<10} {pos_param_str:<10} {chamadas_str}\n")
                    
                    # Tipo de retorno da função
                    f.write(f"\nret_type({table.scope_name}): {table.return_type}\n")
                    f.write("\n" + "=" * 80 + "\n\n")
                
                # Se não há tabelas
                if not self.symbol_table_manager.tables:
                    f.write("Nenhuma tabela de símbolos foi criada.\n")
                        
        except Exception as e:
            print(f"Erro ao gerar arquivo de tabelas de símbolos: {str(e)}")
    
    def parse_declaracao(self):
        let_token = self.match_with_recovery("LET")
        if not let_token:
            return
            
        var_list = self.parse_var_list()
        self.match_with_recovery("COLON")
        type_token = self.parse_type()
        self.match_with_recovery("SEMICOLON")
    
        # ADICIONAR variáveis locais - CORREÇÃO: garantir que as variáveis sejam adicionadas
        if self.symbol_table_manager.current_table:
            for var_name in var_list:
                var_symbol = Symbol(
                    var_name,
                    type_token.value.lower() if type_token else "unknown",
                    False,  # is_param = False
                    let_token.line,
                    self.current_function,
                    -1  # pos_param = -1 para variáveis
                )
                success = self.symbol_table_manager.current_table.add_symbol(var_symbol)
                if not success:
                    self.error(f"Variável '{var_name}' já declarada")
    
    def parse_var_list(self):
        var_names = []
        var_name_token = self.match_with_recovery("ID")
        if var_name_token:
            var_names.append(var_name_token.value)
            # Processar variáveis adicionais
            while self.current_token and self.current_token.type == "COMMA":
                self.match_with_recovery("COMMA")
                next_var = self.match_with_recovery("ID")
                if next_var:
                    var_names.append(next_var.value)
        return var_names
    
    def parse_comando(self):
        if self.current_token and self.current_token.type == "ID":
            var_name = self.current_token.value
            
            # Lookahead para determinar se é atribuição ou chamada
            next_pos = self.pos + 1
            if next_pos < len(self.tokens) and self.tokens[next_pos].type == "ASSIGN":
                # É atribuição
                if not self._is_declared(var_name):
                    self.error(f"Variável '{var_name}' não declarada")
                self.match_with_recovery("ID")
                self.match_with_recovery("ASSIGN")
                self.parse_expr()
                self.match_with_recovery("SEMICOLON")
                
            elif next_pos < len(self.tokens) and self.tokens[next_pos].type == "LBRACKET":
                # É chamada de função
                self.match_with_recovery("ID")
                self.match_with_recovery("LBRACKET")
                args = self.parse_lista_args()
                self.match_with_recovery("RBRACKET")
                self.match_with_recovery("SEMICOLON")
                
                # REGISTRAR chamada de função
                self._register_function_call(var_name, len(args), args)
                
            else:
                self.error("Esperado '=' ou '(' após identificador")
                self.sync_to(["SEMICOLON"])
        elif self.current_token and self.current_token.type == "IF":
            self.parse_comando_se()
        elif self.current_token and self.current_token.type == "WHILE":
            self.parse_comando_while()
        elif self.current_token and self.current_token.type == "PRINTLN":
            self.parse_comando_println()
        elif self.current_token and self.current_token.type == "RETURN":
            self.parse_comando_return()
        else:
            self.error("Comando inválido")
            self.sync_to(["SEMICOLON", "RBRACE"])
    
    def _is_declared(self, name):
        """Verifica se um identificador foi declarado em qualquer escopo acessível"""
        if self.symbol_table_manager.current_table:
            return self.symbol_table_manager.current_table.lookup(name) is not None
        return False
    
    def _register_function_call(self, function_name, num_args, args):
        """Registra uma chamada de função na tabela de símbolos"""
        if not self.symbol_table_manager.current_table:
            return
            
        # Verifica se a função já está na tabela
        symbol = self.symbol_table_manager.current_table.lookup(function_name)
        if not symbol:
            # Cria novo símbolo para função chamada
            symbol = Symbol(function_name, "function", False, 
                          self.current_token.line if self.current_token else 0,
                          self.current_function)
            self.symbol_table_manager.current_table.add_symbol(symbol)
        
        # Cria registro de chamada
        call_register = FunctionRegister(function_name, num_args, args)
        symbol.add_call_ref(call_register)
    
    def parse_comando_se(self):
        """ComandoSe → if Expr Bloco ComandoSenao"""
        self.match_with_recovery("IF")
        self.parse_expr()  
        self.parse_bloco()
        self.parse_comando_senao()
    
    def parse_comando_senao(self):
        """ComandoSenao → else ComandoSe | ε"""
        if self.current_token and self.current_token.type == "ELSE":
            self.match_with_recovery("ELSE")
            # Pode ser outro if ou um bloco simples
            if self.current_token and self.current_token.type == "IF":
                self.parse_comando_se()
            else:
                self.parse_bloco()
        # Senão, é ε - não faz nada
    
    def parse_comando_while(self):
        """ComandoWhile → while Expr Bloco"""
        self.match_with_recovery("WHILE")
        self.parse_expr()
        self.parse_bloco()
        
    def parse_comando_println(self):
        self.match_with_recovery("PRINTLN")
        self.match_with_recovery("LBRACKET")
        self.match_with_recovery("FMT_STRING")
        self.match_with_recovery("COMMA")
        # Parsear expressão em vez de lista de argumentos
        self.parse_expr()
        self.match_with_recovery("RBRACKET")
        self.match_with_recovery("SEMICOLON")
        
        # Registrar chamada do println
        self._register_function_call("println", 1, [])
    
    def parse_comando_return(self):
        self.match_with_recovery("RETURN")
        self.parse_expr()
        self.match_with_recovery("SEMICOLON")
    
    def parse_lista_args(self):
        args = []
        if self.current_token and self.current_token.type in ["ID", "INT_CONST", "FLOAT_CONST", "CHAR_LITERAL", "LBRACKET"]:
            arg_value = self._parse_arg_value()
            if arg_value is not None:
                args.append(arg_value)
            args.extend(self.parse_lista_args2())
        return args

    def parse_lista_args2(self):
        args = []
        while self.current_token and self.current_token.type == "COMMA":
            self.match_with_recovery("COMMA")
            arg_value = self._parse_arg_value()
            if arg_value is not None:
                args.append(arg_value)
        return args

    def _parse_arg_value(self):
        """Extrai o valor do argumento (lexema) - SUPORTA CHAMADAS DE FUNÇÃO ANINHADAS"""
        if self.current_token is None:
            return None
            
        if self.current_token.type == "ID":
            # Lookahead para ver se é chamada de função
            next_pos = self.pos + 1
            if next_pos < len(self.tokens) and self.tokens[next_pos].type == "LBRACKET":
                # É chamada de função - processar recursivamente
                func_name = self.current_token.value
                self.match_with_recovery("ID")
                self.match_with_recovery("LBRACKET")
                inner_args = self.parse_lista_args()
                self.match_with_recovery("RBRACKET")
                
                # Registrar a chamada de função interna
                self._register_function_call(func_name, len(inner_args), inner_args)
                
                return f"{func_name}({', '.join(inner_args)})"
            else:
                # Apenas uma variável
                token = self.match_with_recovery("ID")
                return token.value if token else None
                
        elif self.current_token.type in ["INT_CONST", "FLOAT_CONST", "CHAR_LITERAL"]:
            token = self.match_with_recovery(self.current_token.type)
            return token.value if token else None
            
        elif self.current_token.type == "LBRACKET":
            # Expressão entre parênteses
            self.match_with_recovery("LBRACKET")
            # Para simplificar, vamos processar apenas uma expressão simples
            if self.current_token and self.current_token.type in ["ID", "INT_CONST", "FLOAT_CONST", "CHAR_LITERAL"]:
                arg_value = self._parse_arg_value()
                self.match_with_recovery("RBRACKET")
                return f"({arg_value})" if arg_value else None
            else:
                self.match_with_recovery("RBRACKET")
                return None
        else:
            return None
    
    def parse_arg(self):
        if self.current_token and self.current_token.type == "ID":
            # Lookahead para ver se é chamada de função
            next_pos = self.pos + 1
            if next_pos < len(self.tokens) and self.tokens[next_pos].type == "LBRACKET":
                self.match_with_recovery("ID")
                self.match_with_recovery("LBRACKET")
                self.parse_lista_args()
                self.match_with_recovery("RBRACKET")
            else:
                # Apenas uma variável
                self.match_with_recovery("ID")
        elif self.current_token and self.current_token.type in ["INT_CONST", "FLOAT_CONST", "CHAR_LITERAL"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado ID, constante ou literal")
            if self.current_token:
                self.advance()
    
    def parse_chamada_funcao(self):
        if self.current_token and self.current_token.type == "LBRACKET":
            self.match_with_recovery("LBRACKET")
            self.parse_lista_args()
            self.match_with_recovery("RBRACKET")
    
    # ========== EXPRESSÕES ==========
    def parse_expr(self):
        """Expr → Rel ExprOpc"""
        self.parse_rel()
        self.parse_expr_opc()

    def parse_expr_opc(self):
        """ExprOpc → OpIgual Rel ExprOpc | ε"""
        if self.current_token and self.current_token.type in ["EQ", "NE"]:
            self.parse_op_igual()
            self.parse_rel()
            self.parse_expr_opc()
        # Senão, é ε

    def parse_op_igual(self):
        if self.current_token and self.current_token.type in ["EQ", "NE"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado '==' ou '!='")

    def parse_rel(self):
        """Rel → Adicao RelOpc"""
        self.parse_adicao()
        self.parse_rel_opc()

    def parse_rel_opc(self):
        """RelOpc → OpRel Adicao RelOpc | ε"""
        if self.current_token and self.current_token.type in ["LT", "LE", "GT", "GE"]:
            self.parse_op_rel()
            self.parse_adicao()
            self.parse_rel_opc()
        # Senão, é ε

    def parse_op_rel(self):
        if self.current_token and self.current_token.type in ["LT", "LE", "GT", "GE"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado operador relacional")

    def parse_adicao(self):
        """Adicao → Termo AdicaoOpc"""
        self.parse_termo()
        self.parse_adicao_opc()

    def parse_adicao_opc(self):
        """AdicaoOpc → OpAdicao Termo AdicaoOpc | ε"""
        while self.current_token and self.current_token.type in ["PLUS", "MINUS"]:
            self.parse_op_adicao()
            self.parse_termo()

    def parse_op_adicao(self):
        if self.current_token and self.current_token.type in ["PLUS", "MINUS"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado '+' ou '-'")

    def parse_termo(self):
        """Termo → Fator TermoOpc"""
        self.parse_fator()
        self.parse_termo_opc()

    def parse_termo_opc(self):
        """TermoOpc → OpMult Fator TermoOpc | ε"""
        while self.current_token and self.current_token.type in ["MULT", "DIV"]:
            self.parse_op_mult()
            self.parse_fator()

    def parse_op_mult(self):
        if self.current_token and self.current_token.type in ["MULT", "DIV"]:
            self.match_with_recovery(self.current_token.type)
        else:
            self.error("Esperado '*' ou '/'")

    def parse_fator(self):
        """Fator → ID ChamadaFuncao | CONST | LITERAL | ( Expr )"""
        if self.current_token and self.current_token.type == "ID":
            self.match_with_recovery("ID")
            self.parse_chamada_funcao()
        elif self.current_token and self.current_token.type in ["INT_CONST", "FLOAT_CONST", "CHAR_LITERAL"]:
            self.match_with_recovery(self.current_token.type)
        elif self.current_token and self.current_token.type == "LBRACKET":
            self.match_with_recovery("LBRACKET")
            self.parse_expr()
            self.match_with_recovery("RBRACKET")
        else:
            self.error("Esperado ID, constante, literal ou '('")
            if self.current_token:
                self.advance()

In [13]:
#class symbol

class Symbol:
    def __init__(self, name, datatype, is_param, line, scope, pos_param=-1):
        self.name = name          # nome do identificador
        self.datatype = datatype  # 'int', 'float', 'char', 'function'
        self.is_param = is_param  # True para parâmetros, False para variáveis
        self.line = line          # linha de declaração
        self.scope = scope        # escopo/função pertence
        self.pos_param = pos_param  # -1 para variáveis, 0,1,2... para parâmetros
        self.call_refs = []       # lista de registros de chamada

    def add_call_ref(self, call_register):
        self.call_refs.append(call_register)

In [14]:
#class functionregister

class FunctionRegister:
    def __init__(self, name, num_args, args):
        self.name = name          # nome da função chamada
        self.num_args = num_args  # número de argumentos
        self.args = args          # lista de argumentos (lexemas)

In [15]:
#class symbol table

class SymbolTable:
    def __init__(self, scope_name):
        self.scope_name = scope_name  # nome da função
        self.return_type = "void"     # tipo de retorno (padrão void)
        self.symbols = {}             # dicionário nome -> Symbol
    
    def add_symbol(self, symbol):
        if symbol.name in self.symbols:
            return False
        self.symbols[symbol.name] = symbol
        return True
    
    def lookup(self, name):
        return self.symbols.get(name)
    
    def set_return_type(self, return_type):
        self.return_type = return_type

In [16]:
#class symbol table manager

class SymbolTableManager:
    def __init__(self):
        self.tables = []
        self.current_table = None
    
    def enter_scope(self, function_name):
        new_table = SymbolTable(function_name)
        self.tables.append(new_table)
        self.current_table = new_table
        return new_table
    
    def exit_scope(self):
        self.current_table = None
    
    def to_dict(self):
        return {
            "tabelas": [
                {
                    "escopo": table.scope_name,
                    "ret_type": table.return_type,
                    "simbolos": [
                        {
                            "nome": sym.name,
                            "tipo": sym.datatype,
                            "categoria": "parametro" if sym.is_param else "variavel",
                            "linha": sym.line,
                            "pos_param": sym.pos_param,
                            "chamadas": [
                                {
                                    "nome": call.name,
                                    "num_args": call.num_args,
                                    "args": call.args
                                }
                                for call in sym.call_refs
                            ]
                        }
                        for sym in table.symbols.values()
                    ]
                }
                for table in self.tables
            ]
        }
    
    def to_table_string(self):
        """Retorna uma string com a tabela de símbolos formatada como tabela textual."""
        lines = []
        for table in self.tables:
            lines.append(f"Tabela de símbolos para a função {table.scope_name}:")
            lines.append("| chave | name | datatype | is_param | pos_param | call_refs |")
            lines.append("|-------|------|----------|----------|-----------|-----------|")
            for symbol_name, symbol in table.symbols.items():
                # Formatando call_refs: vamos listar os nomes das chamadas
                call_refs_str = "NULL"
                if symbol.call_refs:
                    # Vamos listar os nomes das funções chamadas
                    call_names = [call.name for call in symbol.call_refs]
                    call_refs_str = ", ".join(call_names)
                lines.append(f"| {symbol_name} | {symbol.name} | {symbol.datatype} | {symbol.is_param} | {symbol.pos_param} | {call_refs_str} |")
            lines.append(f"ret_type({table.scope_name}): {table.return_type}")
            lines.append("")  # linha em branco entre tabelas
        return "\n".join(lines)

<h2>Testes do Parser e da tabela de Simbolos</h2>
<h4>Aqui executamos os testes referentes a cada arquivo base para erros sintaticos e tabelas de simbolos</h4>

In [17]:
#teste calculadora.p - Tabela de Símbolos Formatada
entrada = "codigos base da linguagem p/calculadora.p"
saida_syntax = "saidas do analisador sintatico/calculadora_syntax.txt"
saida_tabelas = "tabelas de simbolos/calculadora_tabelas.txt"

print("TESTANDO: calculadora.p")
print("=" * 60)

try:
    with open(entrada, "r", encoding="utf-8") as f:
        codigo = f.read()
    
    lexer = Lexer(codigo)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    parser.parse("calculadora.p")
    
    # Mostrar resultado
    erros_count = len(parser.errors)
    if erros_count == 0:
        print("ANÁLISE SINTÁTICA: VÁLIDA")
    else:
        print(f"ANÁLISE SINTÁTICA: {erros_count} erro(s)")
        for erro in parser.errors:
            print(f"   {erro}")
    
    # Mostrar tabelas de símbolos formatadas no console
    print("\nTABELAS DE SÍMBOLOS FORMATADAS:")
    print("=" * 60)
    for table in parser.symbol_table_manager.tables:
        print(f"\nTABELA DE SÍMBOLOS: {table.scope_name}")
        print("-" * 60)
        print(f"{'Chave':<12} {'Nome':<12} {'Tipo':<10} {'Parâmetro':<9} {'Pos_Param':<9} {'Chamadas'}")
        print("-" * 60)
        
        for symbol_name, symbol in table.symbols.items():
            is_param_str = "Sim" if symbol.is_param else "Não"
            pos_param_str = str(symbol.pos_param) if symbol.is_param else "-1"
            
            # Formatar chamadas de função
            chamadas_str = "NULL"
            if symbol.call_refs:
                chamadas_list = []
                for call in symbol.call_refs:
                    chamadas_list.append(f"{call.name}({', '.join(call.args)})")
                chamadas_str = ", ".join(chamadas_list)
            
            print(f"{symbol_name:<12} {symbol.name:<12} {symbol.datatype:<10} {is_param_str:<9} {pos_param_str:<9} {chamadas_str}")
        
        print(f"\nret_type({table.scope_name}): {table.return_type}")
    
    print(f"\nArquivos gerados:")
    print(f"   - {saida_syntax}")
    print(f"   - {saida_tabelas}")
    
except Exception as e:
    print(f"ERRO: {e}")

TESTANDO: calculadora.p
ANÁLISE SINTÁTICA: VÁLIDA

TABELAS DE SÍMBOLOS FORMATADAS:

TABELA DE SÍMBOLOS: calculadora
------------------------------------------------------------
Chave        Nome         Tipo       Parâmetro Pos_Param Chamadas
------------------------------------------------------------
calculadora  calculadora  function   Não       -1        NULL
op           op           char       Sim       0         NULL
x            x            float      Sim       1         NULL
y            y            float      Sim       2         NULL

ret_type(calculadora): float

TABELA DE SÍMBOLOS: main
------------------------------------------------------------
Chave        Nome         Tipo       Parâmetro Pos_Param Chamadas
------------------------------------------------------------
main         main         function   Não       -1        NULL
a            a            float      Não       -1        NULL
b            b            float      Não       -1        NULL
println      print

In [18]:
#teste lexical_error.p - Tabela de Símbolos Formatada
entrada = "codigos base da linguagem p/lexical_error.p"
saida_syntax = "saidas do analisador sintatico/lexical_error_syntax.txt"
saida_tabelas = "tabelas de simbolos/lexical_error_tabelas.txt"

print("TESTANDO: lexical_error.p")
print("=" * 60)

try:
    with open(entrada, "r", encoding="utf-8") as f:
        codigo = f.read()
    
    lexer = Lexer(codigo)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    parser.parse("lexical_error.p")
    
    # Mostrar resultado
    erros_count = len(parser.errors)
    if erros_count == 0:
        print("ANÁLISE SINTÁTICA: VÁLIDA")
    else:
        print(f"ANÁLISE SINTÁTICA: {erros_count} erro(s)")
        for erro in parser.errors:
            print(f"   {erro}")
    
    # Mostrar tabelas de símbolos formatadas no console
    print("\nTABELAS DE SÍMBOLOS FORMATADAS:")
    print("=" * 60)
    for table in parser.symbol_table_manager.tables:
        print(f"\nTABELA DE SÍMBOLOS: {table.scope_name}")
        print("-" * 60)
        print(f"{'Chave':<12} {'Nome':<12} {'Tipo':<10} {'Parâmetro':<9} {'Pos_Param':<9} {'Chamadas'}")
        print("-" * 60)
        
        for symbol_name, symbol in table.symbols.items():
            is_param_str = "Sim" if symbol.is_param else "Não"
            pos_param_str = str(symbol.pos_param) if symbol.is_param else "-1"
            
            # Formatar chamadas de função
            chamadas_str = "NULL"
            if symbol.call_refs:
                chamadas_list = []
                for call in symbol.call_refs:
                    chamadas_list.append(f"{call.name}({', '.join(call.args)})")
                chamadas_str = ", ".join(chamadas_list)
            
            print(f"{symbol_name:<12} {symbol.name:<12} {symbol.datatype:<10} {is_param_str:<9} {pos_param_str:<9} {chamadas_str}")
        
        print(f"\nret_type({table.scope_name}): {table.return_type}")
    
    print(f"\nArquivos gerados:")
    print(f"   - {saida_syntax}")
    print(f"   - {saida_tabelas}")
    
except Exception as e:
    print(f"ERRO: {e}")

TESTANDO: lexical_error.p
Erro léxico: caractere inesperado '[' na linha 3
Erro léxico: caractere inesperado ']' na linha 3
Erro léxico: caractere inesperado '$' na linha 5
Erro léxico: caractere inesperado '&' na linha 6
Erro de sintaxe: Linha 6: Esperado 'RBRACKET', encontrado 'FMT_STRING'
  [RECUPERAÇÃO] Pulando token: Token(FMT_STRING, '"{}"', line 6)
  [RECUPERAÇÃO] Pulando token: Token(COMMA, ',', line 6)
  [RECUPERAÇÃO] Pulando token: Token(ID, 'a', line 6)
  [RECUPERAÇÃO] Pulando token: Token(RBRACKET, ')', line 6)
  [RECUPERAÇÃO] Sincronizado em: Token(SEMICOLON, ';', line 6)
ANÁLISE SINTÁTICA: 1 erro(s)
   Linha 6: Esperado 'RBRACKET', encontrado 'FMT_STRING'

TABELAS DE SÍMBOLOS FORMATADAS:

TABELA DE SÍMBOLOS: main
------------------------------------------------------------
Chave        Nome         Tipo       Parâmetro Pos_Param Chamadas
------------------------------------------------------------
main         main         function   Não       -1        NULL
a            

In [19]:
#teste loop_simples.p - Tabela de Símbolos Formatada
entrada = "codigos base da linguagem p/loop_simples.p"
saida_syntax = "saidas do analisador sintatico/loop_simples_syntax.txt"
saida_tabelas = "tabelas de simbolos/loop_simples_tabelas.txt"

print("TESTANDO: loop_simples.p")
print("=" * 60)

try:
    with open(entrada, "r", encoding="utf-8") as f:
        codigo = f.read()
    
    lexer = Lexer(codigo)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    parser.parse("loop_simples.p")
    
    # Mostrar resultado
    erros_count = len(parser.errors)
    if erros_count == 0:
        print("ANÁLISE SINTÁTICA: VÁLIDA")
    else:
        print(f"ANÁLISE SINTÁTICA: {erros_count} erro(s)")
        for erro in parser.errors:
            print(f"   {erro}")
    
    # Mostrar tabelas de símbolos formatadas no console
    print("\nTABELAS DE SÍMBOLOS FORMATADAS:")
    print("=" * 60)
    for table in parser.symbol_table_manager.tables:
        print(f"\nTABELA DE SÍMBOLOS: {table.scope_name}")
        print("-" * 60)
        print(f"{'Chave':<12} {'Nome':<12} {'Tipo':<10} {'Parâmetro':<9} {'Pos_Param':<9} {'Chamadas'}")
        print("-" * 60)
        
        for symbol_name, symbol in table.symbols.items():
            is_param_str = "Sim" if symbol.is_param else "Não"
            pos_param_str = str(symbol.pos_param) if symbol.is_param else "-1"
            
            # Formatar chamadas de função
            chamadas_str = "NULL"
            if symbol.call_refs:
                chamadas_list = []
                for call in symbol.call_refs:
                    chamadas_list.append(f"{call.name}({', '.join(call.args)})")
                chamadas_str = ", ".join(chamadas_list)
            
            print(f"{symbol_name:<12} {symbol.name:<12} {symbol.datatype:<10} {is_param_str:<9} {pos_param_str:<9} {chamadas_str}")
        
        print(f"\nret_type({table.scope_name}): {table.return_type}")
    
    print(f"\nArquivos gerados:")
    print(f"   - {saida_syntax}")
    print(f"   - {saida_tabelas}")
    
except Exception as e:
    print(f"ERRO: {e}")

TESTANDO: loop_simples.p
ANÁLISE SINTÁTICA: VÁLIDA

TABELAS DE SÍMBOLOS FORMATADAS:

TABELA DE SÍMBOLOS: main
------------------------------------------------------------
Chave        Nome         Tipo       Parâmetro Pos_Param Chamadas
------------------------------------------------------------
main         main         function   Não       -1        NULL
i            i            int        Não       -1        NULL
println      println      function   Não       -1        println()

ret_type(main): void

Arquivos gerados:
   - saidas do analisador sintatico/loop_simples_syntax.txt
   - tabelas de simbolos/loop_simples_tabelas.txt


In [20]:
#teste media.p - Tabela de Símbolos Formatada
entrada = "codigos base da linguagem p/media.p"
saida_syntax = "saidas do analisador sintatico/media_syntax.txt"
saida_tabelas = "tabelas de simbolos/media_tabelas.txt"

print("TESTANDO: media.p")
print("=" * 60)

try:
    with open(entrada, "r", encoding="utf-8") as f:
        codigo = f.read()
    
    lexer = Lexer(codigo)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    parser.parse("media.p")
    
    # Mostrar resultado
    erros_count = len(parser.errors)
    if erros_count == 0:
        print("ANÁLISE SINTÁTICA: VÁLIDA")
    else:
        print(f"ANÁLISE SINTÁTICA: {erros_count} erro(s)")
        for erro in parser.errors:
            print(f"   {erro}")
    
    # Mostrar tabelas de símbolos formatadas no console
    print("\nTABELAS DE SÍMBOLOS FORMATADAS:")
    print("=" * 60)
    for table in parser.symbol_table_manager.tables:
        print(f"\nTABELA DE SÍMBOLOS: {table.scope_name}")
        print("-" * 60)
        print(f"{'Chave':<12} {'Nome':<12} {'Tipo':<10} {'Parâmetro':<9} {'Pos_Param':<9} {'Chamadas'}")
        print("-" * 60)
        
        for symbol_name, symbol in table.symbols.items():
            is_param_str = "Sim" if symbol.is_param else "Não"
            pos_param_str = str(symbol.pos_param) if symbol.is_param else "-1"
            
            # Formatar chamadas de função
            chamadas_str = "NULL"
            if symbol.call_refs:
                chamadas_list = []
                for call in symbol.call_refs:
                    chamadas_list.append(f"{call.name}({', '.join(call.args)})")
                chamadas_str = ", ".join(chamadas_list)
            
            print(f"{symbol_name:<12} {symbol.name:<12} {symbol.datatype:<10} {is_param_str:<9} {pos_param_str:<9} {chamadas_str}")
        
        print(f"\nret_type({table.scope_name}): {table.return_type}")
    
    print(f"\nArquivos gerados:")
    print(f"   - {saida_syntax}")
    print(f"   - {saida_tabelas}")
    
except Exception as e:
    print(f"ERRO: {e}")

TESTANDO: media.p
ANÁLISE SINTÁTICA: VÁLIDA

TABELAS DE SÍMBOLOS FORMATADAS:

TABELA DE SÍMBOLOS: main
------------------------------------------------------------
Chave        Nome         Tipo       Parâmetro Pos_Param Chamadas
------------------------------------------------------------
main         main         function   Não       -1        NULL
a            a            float      Não       -1        NULL
b            b            float      Não       -1        NULL
c            c            float      Não       -1        NULL
media        media        float      Não       -1        NULL
println      println      function   Não       -1        println()

ret_type(main): void

Arquivos gerados:
   - saidas do analisador sintatico/media_syntax.txt
   - tabelas de simbolos/media_tabelas.txt


In [21]:
#teste soma.p - Tabela de Símbolos Formatada
entrada = "codigos base da linguagem p/soma.p"
saida_syntax = "saidas do analisador sintatico/soma_syntax.txt"
saida_tabelas = "tabelas de simbolos/soma_tabelas.txt"

print("TESTANDO: soma.p")
print("=" * 60)

try:
    with open(entrada, "r", encoding="utf-8") as f:
        codigo = f.read()
    
    lexer = Lexer(codigo)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    parser.parse("soma.p")
    
    # Mostrar resultado
    erros_count = len(parser.errors)
    if erros_count == 0:
        print("ANÁLISE SINTÁTICA: VÁLIDA")
    else:
        print(f"ANÁLISE SINTÁTICA: {erros_count} erro(s)")
        for erro in parser.errors:
            print(f"   {erro}")
    
    # Mostrar tabelas de símbolos formatadas no console
    print("\nTABELAS DE SÍMBOLOS FORMATADAS:")
    print("=" * 60)
    for table in parser.symbol_table_manager.tables:
        print(f"\nTABELA DE SÍMBOLOS: {table.scope_name}")
        print("-" * 60)
        print(f"{'Chave':<12} {'Nome':<12} {'Tipo':<10} {'Parâmetro':<9} {'Pos_Param':<9} {'Chamadas'}")
        print("-" * 60)
        
        for symbol_name, symbol in table.symbols.items():
            is_param_str = "Sim" if symbol.is_param else "Não"
            pos_param_str = str(symbol.pos_param) if symbol.is_param else "-1"
            
            # Formatar chamadas de função
            chamadas_str = "NULL"
            if symbol.call_refs:
                chamadas_list = []
                for call in symbol.call_refs:
                    chamadas_list.append(f"{call.name}({', '.join(call.args)})")
                chamadas_str = ", ".join(chamadas_list)
            
            print(f"{symbol_name:<12} {symbol.name:<12} {symbol.datatype:<10} {is_param_str:<9} {pos_param_str:<9} {chamadas_str}")
        
        print(f"\nret_type({table.scope_name}): {table.return_type}")
    
    print(f"\nArquivos gerados:")
    print(f"   - {saida_syntax}")
    print(f"   - {saida_tabelas}")
    
except Exception as e:
    print(f"ERRO: {e}")

TESTANDO: soma.p
ANÁLISE SINTÁTICA: VÁLIDA

TABELAS DE SÍMBOLOS FORMATADAS:

TABELA DE SÍMBOLOS: soma
------------------------------------------------------------
Chave        Nome         Tipo       Parâmetro Pos_Param Chamadas
------------------------------------------------------------
soma         soma         function   Não       -1        NULL
x            x            int        Sim       0         NULL
y            y            int        Sim       1         NULL

ret_type(soma): int

TABELA DE SÍMBOLOS: main
------------------------------------------------------------
Chave        Nome         Tipo       Parâmetro Pos_Param Chamadas
------------------------------------------------------------
main         main         function   Não       -1        NULL
a            a            int        Não       -1        NULL
b            b            int        Não       -1        NULL
c            c            int        Não       -1        NULL
println      println      function   Não 